In [1]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
import tensorflow.python.keras
from tensorflow.python.keras import models, layers
from tensorflow.python.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.python.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
# from tensorflow import keras

# from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
# import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.75
config.gpu_options.allow_growth= True
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# Create a session with the above options specified.
# k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 10
l = 40
compression = 0.55
dropout_rate = 0.2

In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [5]:
X_train.shape

(50000, 32, 32, 3)

In [6]:
X_test.shape

(10000, 32, 32, 3)

In [7]:
#pip install --upgrade "tensorflow==1.4" "keras>=2.0"

In [8]:
X_train_mean = np.mean(X_train, axis=(0,1,2))
X_train_std = np.std(X_train, axis=(0,1,2))
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std

In [9]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.SeparableConv2D(int(num_filter*compression), (5,5), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.SeparableConv2D(int(num_filter*compression), (5,5), use_bias=False,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(4,4))(relu)
    #flat = layers.Flatten()(AvgPooling)
    output = layers.SeparableConv2D(num_classes, (5,5), use_bias=False,padding='same', activation='softmax')(AvgPooling)
    return output

In [10]:
import keras.backend as K
K.clear_session()


In [11]:
#pip install --upgrade "tensorflow==1.4" "keras>=2.0"

In [12]:
num_filter = 126

dropout_rate = 0
l = 7

input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.SeparableConv2D(num_filter, (5,5), use_bias=False, padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 32, 32, 126)  453         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 126)  504         separable_conv2d[0][0]           
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 126)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [14]:

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=0.10,
    )
datagen.fit(X_train)

In [15]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau, LearningRateScheduler

In [16]:
#https://machinelearningmastery.com/check-point-deep-learning-models-keras/
filepath="epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint_1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')


In [17]:
reduce_lr_1 = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=4,verbose = 1)

In [18]:
earlystopping_1 = EarlyStopping(monitor='val_loss', patience=40, verbose=1)

In [19]:
callbacks_list = [earlystopping_1,reduce_lr_1,checkpoint_1]

In [20]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [21]:
# reshaping to match with convoultion output layer
y_train_re = np.reshape(y_train, (50000,1,1,10))
y_test_re = np.reshape(y_test, (10000,1,1,10))

In [ ]:
history = model.fit_generator(datagen.flow(X_train, y_train_re, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=100,
                    verbose=1, 
                    validation_data=(X_test, y_test_re),callbacks=callbacks_list)

Epoch 1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
780/781 [============================>.] - ETA: 0s - loss: 1.6015 - acc: 0.4069
Epoch 00001: saving model to epochs:001-val_acc:0.509.hdf5
781/781 [==============================] - 142s 182ms/step - loss: 1.6011 - acc: 0.4070 - val_loss: 1.4417 - val_acc: 0.5087
Epoch 2/100
780/781 [============================>.] - ETA: 0s - loss: 1.1707 - acc: 0.5825
Epoch 00002: saving model to epochs:002-val_acc:0.530.hdf5
781/781 [==============================] - 125s 160ms/step - loss: 1.1703 - acc: 0.5827 - val_loss: 1.5555 - val_acc: 0.5298
Epoch 3/100
780/781 [============================>.] - ETA: 0s - loss: 0.9674 - acc: 0.6601
Epoch 00003: saving model to epochs:003-val_acc:0.590.hdf5
781/781 [==============================] - 125s 160ms/step - loss: 0.9678 - acc: 0.6600 - val_loss: 1.3740 - val_acc: 0.5901
Epoch 4/100
780/781 [============================>.] - ETA: 0s - loss: 0.8379 

In [28]:
#Connection got disconnected, so resuminf the traimimg process from epoch27

In [22]:
checkpoint_path = "epochs:026-val_acc:0.899.hdf5" 
model = load_model(checkpoint_path)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [24]:
model.fit_generator(datagen.flow(X_train, y_train_re, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=100,
                    verbose=1, 
                    validation_data=(X_test, y_test_re),callbacks=callbacks_list, initial_epoch=26)

Epoch 27/100
780/781 [============================>.] - ETA: 0s - loss: 0.1915 - acc: 0.9328
Epoch 00027: saving model to epochs:027-val_acc:0.902.hdf5
781/781 [==============================] - 144s 184ms/step - loss: 0.1913 - acc: 0.9328 - val_loss: 0.3185 - val_acc: 0.9017
Epoch 28/100
780/781 [============================>.] - ETA: 0s - loss: 0.1796 - acc: 0.9379
Epoch 00028: saving model to epochs:028-val_acc:0.902.hdf5
781/781 [==============================] - 127s 162ms/step - loss: 0.1795 - acc: 0.9379 - val_loss: 0.3165 - val_acc: 0.9021
Epoch 29/100
780/781 [============================>.] - ETA: 0s - loss: 0.1752 - acc: 0.9392
Epoch 00029: saving model to epochs:029-val_acc:0.905.hdf5
781/781 [==============================] - 127s 162ms/step - loss: 0.1752 - acc: 0.9391 - val_loss: 0.3142 - val_acc: 0.9052
Epoch 30/100
780/781 [============================>.] - ETA: 0s - loss: 0.1696 - acc: 0.9407
Epoch 00030: saving model to epochs:030-val_acc:0.904.hdf5
781/781 [========

780/781 [============================>.] - ETA: 0s - loss: 0.1367 - acc: 0.9522
Epoch 00055: saving model to epochs:055-val_acc:0.908.hdf5
781/781 [==============================] - 127s 163ms/step - loss: 0.1367 - acc: 0.9522 - val_loss: 0.3102 - val_acc: 0.9082
Epoch 56/100
780/781 [============================>.] - ETA: 0s - loss: 0.1368 - acc: 0.9522
Epoch 00056: saving model to epochs:056-val_acc:0.909.hdf5
781/781 [==============================] - 127s 163ms/step - loss: 0.1368 - acc: 0.9522 - val_loss: 0.3088 - val_acc: 0.9091
Epoch 57/100
780/781 [============================>.] - ETA: 0s - loss: 0.1346 - acc: 0.9527
Epoch 00057: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.

Epoch 00057: saving model to epochs:057-val_acc:0.909.hdf5
781/781 [==============================] - 127s 163ms/step - loss: 0.1347 - acc: 0.9527 - val_loss: 0.3091 - val_acc: 0.9090
Epoch 58/100
780/781 [============================>.] - ETA: 0s - loss: 0.1371 - acc: 0.9536
Epoch 00

In [26]:
# Test the model
score = model.evaluate(X_test, y_test_re, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 7s 652us/sample - loss: 0.3113 - acc: 0.9088
Test loss: 0.31126459009647367
Test accuracy: 0.9088


In [27]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk
